# Boosting Algorithm: Adaptive Boosting Method (AdaBoost)

Rafiq Islam  
2024-12-02

## Introduction

Boosting is a powerful ensemble learning technique that focuses on
improving the performance of weak learners to build a robust predictive
model. <br> <br> Now the question is what the heck is weak learner?
Well, roughly speaking, a statistical learning algorithm is called a
weak learner if it is slightly better than just random guess. In
contrast, a statistical learning algorithm is called a strong learner if
it can be made arbitrarily close to the true value.<br> <br> Unlike
bagging (bootstrap aggregating, e.g. random forest), which builds models
independently, boosting builds models sequentially, where each new model
corrects the errors of its predecessors. This approach ensures that the
ensemble concentrates on the difficult-to-predict instances, making
boosting highly effective for both classification and regression
problems.

### Key Characteristics of Boosting:

1.  **Sequential Model Building:** Boosting builds one model at a time,
    with each model improving upon the errors of the previous one.
2.  **Weight Assignment:** It assigns weights to instances, emphasizing
    misclassified or poorly predicted ones in subsequent iterations.
3.  **Weak to Strong Learners:** The goal of boosting is to combine
    multiple weak learners (models slightly better than random guessing)
    into a strong learner.

------------------------------------------------------------------------

## Adaptive Boosting (AdaBoost)

Adaptive Boosting, or AdaBoost, is one of the earliest and most widely
used boosting algorithms. It was introduced by Freund and Schapire in
1996. AdaBoost combines weak learners, typically decision stumps
(single-level decision trees), to form a strong learner.

#### How AdaBoost Works:

1.  **Initialization:**
    -   Assign an initial weight $w_i^{(1)}$ to each training instance
        $i$, where $w_i^{(1)} = \frac{1}{N}$, and $N$ is the total
        number of training examples.
    -   All instances start with equal weights.
2.  **Iterative Model Training:**
    -   At each iteration $t$, train a weak learner $h_t(x)$ on the
        weighted dataset.

    -   Compute the weighted error rate $\epsilon_t$: $$
        \epsilon_t = \frac{\sum_{i=1}^N w_i^{(t)} \cdot \mathbb{1}(y_i \neq h_t(x_i))}{\sum_{i=1}^N w_i^{(t)}}
        $$

        Here, $\mathbb{1}$ is an indicator function that equals 1 when
        the prediction is incorrect and 0 otherwise.

    -   Calculate the model’s weight $\alpha_t$: $$
        \alpha_t = \frac{1}{2} \ln \left(\frac{1 - \epsilon_t}{\epsilon_t}\right)
        $$ The weight $\alpha_t$ determines the importance of $h_t(x)$
        in the final ensemble. Models with lower error rates receive
        higher weights.
3.  **Update Weights:**
    -   Adjust the weights of the training instances: $$
        w_i^{(t+1)} = w_i^{(t)} \cdot \exp\left(-\alpha_t \cdot y_i \cdot h_t(x_i)\right)
        $$ Instances misclassified by $h_t(x)$ have their weights
        increased, making them more influential in the next iteration.

    -   Normalize the weights to ensure they sum to 1: $$
        w_i^{(t+1)} \leftarrow \frac{w_i^{(t+1)}}{\sum_{j=1}^N w_j^{(t+1)}}
        $$
4.  **Final Model:**
    -   Combine the weak learners into a final strong learner: $$
        H(x) = \text{sign}\left(\sum_{t=1}^T \alpha_t \cdot h_t(x)\right)
        $$ The sign function determines the final class label based on
        the weighted sum of weak learners’ predictions.

------------------------------------------------------------------------

#### Mathematical Derivation and Explanation of AdaBoost

The core idea of AdaBoost is to minimize an exponential loss function:
$$
\mathcal{L} = \sum_{i=1}^N \exp\left(-y_i \cdot F(x_i)\right),
$$ where $F(x_i)$ is the weighted combination of weak learners: $$
F(x_i) = \sum_{t=1}^T \alpha_t \cdot h_t(x_i).
$$

##### 1. **Weight Update Rule:**

-   The exponential term $\exp(-y_i \cdot F(x_i))$ increases for
    misclassified instances ($y_i \neq h_t(x_i)$), assigning them more
    weight in subsequent iterations.

##### 2. **Optimal $\alpha_t$:**

-   The weight $\alpha_t$ is derived to minimize the weighted error
    $\epsilon_t$. A higher value of $\alpha_t$ corresponds to a weak
    learner with better performance.

##### 3. **Boosting as Gradient Descent:**

-   AdaBoost can be interpreted as a stage-wise optimization of the
    exponential loss function. Each iteration reduces the overall loss
    by focusing on misclassified examples.

------------------------------------------------------------------------

### Advantages of AdaBoost:

-   **Simplicity:** Easy to implement with weak learners like decision
    stumps.
-   **No Parameter Tuning:** AdaBoost has fewer hyperparameters to tune
    compared to other boosting methods.
-   **Versatility:** Works well with various types of data and weak
    learners.

### Limitations of AdaBoost:

-   **Sensitivity to Noise:** Outliers can receive disproportionately
    high weights, leading to overfitting.
-   **Weak Learner Dependency:** The performance heavily depends on the
    choice of the weak learner.

------------------------------------------------------------------------

### Closing Thoughts

AdaBoost is a foundational method that inspired more advanced boosting
algorithms like Gradient Boosting and XGBoost. While AdaBoost excels in
simplicity and interpretability, other methods address its limitations
and enhance performance on larger datasets.

In the next post, I will delve into Gradient Boosting, exploring its
mechanics, mathematical foundation, and how it builds on the ideas of
AdaBoost to improve predictive modeling. Stay tuned!